# Solar irradiance on sloping terrain

This module determines the terrain gradient, aspects and solar irradiance on a terrain specified by a 2-D array.

In [ ]:
import math
import numpy as np

In [1]:
def readarr(filename):
    """ Reads the digital elevation data stored in the form 
        of triplets (Easting, Northing, elevation) into
        a Python dictionary resembling a 2-D array,
        but more flexible in that it allows indices to
        store real false Easting/Northing (i.e. indicies
        not necessary starting from 0)."""
    with open(filename) as f:
        elevation = {}
        for line in f:
            x, y, z, *_ = line.split()   # discard remaining contents after z
            x, y, z = int(x), int(y), int(z)    # string --> int
            elevation[(x,y)] = z
        return elevation

In [2]:
elev = readarr('efyrnwy.dem')

In [9]:
elev.keys()
elev[(295000, 315000)]

407

Formulae for the EW gradient:

$$ \text{gradEW} = \frac{\text{elev}[E-\Delta, N] - \text{elev}[E + \Delta, N]}{2 \Delta} $$

the NS gradient:

$$ \text{gradNS} = \frac{\text{elev}[E, N+\Delta] - \text{elev}[E, N - \Delta]}{2 \Delta} $$

the overall slope angle:

$$ \theta = \tan^{-1} \left( \sqrt{\text{gradEW}^2 + \text{gradNS}^2} \right) $$

the aspect:

$$ \phi = 180 + \tan^{-1} \left( -\frac{\text{gradEW}}{\text{gradNS}} \right) $$

In [ ]:
def gradasp(demfile, size):
    """ Takes a 2D DEM and for each cell determines the local 
    gradient and aspect using the method of Zevenbergen 
    and Thorne (1987). Prints out two files, each 
    containing the coordinates of the cell concerned 
    and either (i) its gradient or (ii) its aspect.
    
    Parameters
    ----------
    `demfile`   Name of DEM input file
    `size`      DEM cell size (metres)

    Variables
    ---------
    min_E      eastern-most Easting in DEM
    max_E      western-most Easting in DEM
    min_N      southern-most Northing in DEM
    min_N      northern-most Northing in DEM
    this_E     Easting of current cell
    this_N     Northing of current cell
    gradNS     Gradient of current cell measured North-South
    gradEW     Gradient of current cell measured East-West
    gradient   Gradient of current cell
    aspect     Aspect of current cell
    elevation  2D array storing the DEM
    rad2deg    Conversion factor - radians to degrees. """
    
    
    PI = math.pi
    rad2deg = 360.0/(2*PI)

    with open(demfile) as f:
        first_line = True
        for line in f:
            E, N, z, *_ = line.split()   # discard remaining contents after z
            E, N, z = int(E), int(N), int(z)    # string --> int
            
            # Initialize minimum and maximum Easting and Northing 
            # of the DEM
            if first_line:
                min_E, max_E, min_N, max_N = E, E, N, N
            
            if E < min_E: min_E = E   # Determine minimum and
            if E > max_E: max_E = E   # maximum Eastings and
            if N < min_N: min_N = N   # Northings (i.e., limits)
            if N > max_N: max_N = N   # of DEM
            
    # Pass across DEM by Easting
    with open("aspect.dem", "w") as f_asp, open("gradient.dem", "w") as f_grad:
        gradient = {}
        for this_E in range(min_E+size, max_E, size):
        
            # Pass up through DEM by Northing
            for this_N in range(min_N+size, max_N, size):
            
                # Calculate Zevenbergen and Thorne's (1987) parameters
                gradEW = (z[this_E-size,this_N]- \
                        z[this_E+size,this_N])/(2*size);
                gradNS = (z[this_E,this_N+size]- \
                        z[this_E,this_N-size])/(2*size);
                
                # Calculate the gradient (degrees)
                gradient[(this_E,this_N)]=rad2deg*\
                        (gradNS*gradNS + gradEW*gradEW)**0.5;
                # Print this value out to a file
                f_grad.write("%6i %6i %5.2f\n", this_E, this_N, 
                            gradient[this_E,this_N])
                
                # Calculate the aspect (degrees)
                aspect = 180 + rad2deg * math.atan2(-gradEW,gradNS);
                # Print this value out to a file
                f_asp.write("%6i %6i %3i\n", this_E, this_N, 
                            aspect)
